In [1]:
require 'nn'
require 'nngraph'
Plot = require "itorch.Plot"
lstm = {}
include('utils.lua')
include('LSTM.lua')

In [2]:
-- model hyper-params
emb_dim = 2
sum_rep_dim = 20
class_hidden = 40
class_types = 2
seq_len = 5
-- create a model (in forward order)
emb = nn.LookupTable(10, emb_dim) -- embeddings layer
lstm_net = lstm.LSTM({input_dim=emb_dim, hidden_dim=sum_rep_dim}) -- LSTM layer
classifier = nn.Sequential()  -- classification layer
classifier:add(nn.Linear(sum_rep_dim, class_hidden)) -- classification: linear
classifier:add(nn.Tanh())  -- classification: activation
classifier:add(nn.Linear(class_hidden, class_types)) -- classification: output
classifier:add(nn.LogSoftMax()) -- classification: softmax
criterion = nn.ClassNLLCriterion() -- negative log-likelihood for training
-- the rest is not need (I am going to write SGD myself)
local modules = nn.Parallel()
modules:add(emb)
modules:add(lstm_net)
modules:add(classifier)
params, grad_params = modules:getParameters()

In [3]:
-- Utility functions
function plot_emb(emb, plot)
    local odd_points = emb.weight:index(1, torch.LongTensor{1,3,5,7,9})
    local even_points = emb.weight:index(1, torch.LongTensor{2,4,6,8,10})
    if plot == nil then
        plot = Plot()
    else
        plot._data = {}
    end
    plot:circle(odd_points[{{}, 1}], odd_points[{{}, 2}], 'red', 'even')
    plot:circle(even_points[{{}, 1}], even_points[{{}, 2}], 'blue', 'odd')
    plot:title('Embeddings of numbers 0 - 9')
    plot:legend(true)
    plot:redraw()
    return plot
end
function gen_data(len)
    local inputs = torch.Tensor(len)
    local outputs = torch.Tensor(len)
    for i = 1, len do
        inputs[i] = torch.uniform(1,10)
    end
    inputs:floor()
    local s = 0
    for i = 1, len do
        s = s + inputs[i] - 1
        outputs[i] = (s % 2) + 1
    end
    return inputs, outputs
end
function display(inputs, outputs)
    local s = 0
    for i = 1, inputs:size(1) do
        s = s + inputs[i] - 1
        out_text = 'Even'
        if outputs[i] == 2 then out_text = 'Odd' end
        print(string.format('input %d, sum %d, output %s', inputs[i] - 1, s, out_text))
    end
end

In [4]:
-- Initial embeddings
local _ = plot_emb(emb, nil)

In [5]:
-- SGD hyper-params (this basic version only needs learning rate and number of iterations)
local learning_rate = 0.01
local epochs = 6000
-- Parallization is not helpful here
local num_threads = torch.getnumthreads()
torch.setnumthreads(1)
-- do many iterations
--   \[T]/  Praise the Sun!
--    |_|
--    \ \
for i = 1,epochs do
    -- Reset the network
    lstm_net:forget()
    lstm_net:zeroGradParameters()
    emb:zeroGradParameters()
    local input_seq, output_seq = gen_data(seq_len) -- get inputs and outputs
    local emb_seq = emb:forward(input_seq)          -- get inputs' embeddings
    local sum_rep_seq = lstm_net:forward(emb_seq:t()) -- get representations from LSTM
    local grad_sum_rep_seq = torch.Tensor(sum_rep_dim, seq_len) -- for later
    grad_sum_rep_seq:zero()
    for t = 1,seq_len do -- for each sum (in the sequence)
        classifier:zeroGradParameters() -- reset the classifier
        -- get output log distribution, only need hidden state
        local net_output = classifier:forward(sum_rep_seq[t][2]) 
        local loss = criterion:forward(net_output, output_seq[t]) -- compute loss
        local grad_out = criterion:backward(net_output, output_seq[t]) -- gradients of the loss
        local grad_sum_rep = classifier:backward(sum_rep_seq[t][2], grad_out) -- gradients of the representation
        grad_sum_rep_seq[{{},t}] = grad_sum_rep -- we get the gradient one at a time, so keep it here for later
        classifier:updateParameters(learning_rate) -- update the classifier's parameters
    end
    -- compute embedding gradients (using all of the representation gradients)
    local grad_emb_seq = lstm_net:backward(emb_seq:t(), grad_sum_rep_seq)
    -- compute input gradients 
    -- (this returns nothing, because the input is just index.
    -- But we need to call it to update the gradients accroding to the inputs
    emb:backward(input_seq, grad_emb_seq:t())
    -- update parameters
    lstm_net:updateParameters(learning_rate)
    emb:updateParameters(learning_rate)
end
-- Set num threads back
torch.setnumthreads(num_threads)

In [9]:
-- Testing
lstm_net:forget() -- reset LSTM (go back to step 0)
local input_seq, output_seq = gen_data(seq_len)
local emb_seq = emb:forward(input_seq)
local sum_rep_seq = lstm_net:forward(emb_seq:t())
local output_seq = torch.Tensor(seq_len) -- model's outputs, for later
for t = 1,seq_len do
    -- print('input:')
    -- print(input_seq[t] - 1)
    -- print(sum_rep_seq[t][2])
    local net_output = classifier:forward(sum_rep_seq[t][2])
    -- print(net_output)
    local m, d = torch.max(net_output, 1)
    output_seq[t] = d -- keep the output here
end
-- show the result, is it perfect?
-- Yes: cool, can we make the sequence longer?
-- No: tweak some hyper-parameters
display(input_seq, output_seq) 

input 4, sum 4, output Even	
input 5, sum 9, output Odd	
input 8, sum 17, output Odd	
input 2, sum 19, output Odd	
input 5, sum 24, output Even	


In [10]:
local _ = plot_emb(emb,nil)